## The Impact of Tuition and Financial Aid on Student Retention and Post-Graduation Outcomes
#### Ava Lasater
#### INFO 523 Data Mining

### Import Statements

In [2]:
import pandas as pd

### Data Loading
*Since the data file was so large, I had to split the data using excel to break it down into files that could be uploaded. Since the files are still large, I will be loading then removing columns before joining the data frames back together.*

In [8]:
# using pandas to upload the data chunks
# need to install openpyxl to read excel files

AK_DE = pd.read_csv('/workspaces/final-project-ava-lasater/data/CollegeScorecardAK_DE.csv')
FL_KS = pd.read_excel('/workspaces/final-project-ava-lasater/data/CollegeScorecardFL_KS.xlsx')
KY_MN = pd.read_excel('/workspaces/final-project-ava-lasater/data/CollegeScorecardKY_MN.xlsx')
MO_NJ = pd.read_excel('/workspaces/final-project-ava-lasater/data/CollegeScorecardMO_NJ.xlsx')
NM_OR = pd.read_excel('/workspaces/final-project-ava-lasater/data/CollegeScorecardNM_OR.xlsx')
PA_TX = pd.read_excel('/workspaces/final-project-ava-lasater/data/CollegeScorecardPA_TX.xlsx')
UT_WY = pd.read_excel('/workspaces/final-project-ava-lasater/data/CollegeScorecardUT_WY.xlsx')

KeyboardInterrupt: 

In [ ]:
# performing some basic data cleaning
# I will be deleting columns that have Pell Grant data because I want to focus on other aspects of college data
# will be doing this for each chunk of data so that it is less intesive to join the data later

print(AK_DE.shape)
AK_DE_NP = AK_DE.drop(AK_DE.filter(regex='PELL', axis=1).columns, axis=1)
print(AK_DE_NP.shape)
FL_KS_NP = FL_KS.drop(FL_KS.filter(regex='PELL', axis=1).columns, axis=1)
KY_MN_NP = KY_MN.drop(KY_MN.filter(regex='PELL', axis=1).columns, axis=1)
MO_NJ_NP = MO_NJ.drop(MO_NJ.filter(regex='PELL', axis=1).columns, axis=1)
NM_OR_NP = NM_OR.drop(NM_OR.filter(regex='PELL', axis=1).columns, axis=1)
PA_TX_NP = PA_TX.drop(PA_TX.filter(regex='PELL', axis=1).columns, axis=1)
UT_WY_NP = UT_WY.drop(UT_WY.filter(regex='PELL', axis=1).columns, axis=1)

